In [ ]:
import numpy as np
import pandas as pd
import os
import cv2

In [ ]:
# params
exam = '030703_07_2019_08_09_08'
folder_exams = 'Frames'
folder_labels = 'Labels'

full_exam = False

In [ ]:
path_original_exams = '/media/marcos/Dados/Projects/Datasets/Exams'
path_exams = '/media/marcos/Dados/Projects/Results/PupilDeep/{}'.format(folder_exams)
path_labels = '/media/marcos/Dados/Projects/Results/PupilDeep/{}'.format(folder_labels)
path_out = '{}/exams'.format(path_exams)

print(path_original_exams, path_exams, path_labels, path_out)

In [ ]:
paths_frames = []
if exam == '':
    if os.path.exists(path_out):
        frames_exists = list(map(lambda x: x.replace('.mp4', ''), os.listdir(path_out)))
        frames_total = ['{}'.format(x) for x in os.listdir(path_exams) if ('.' not in x) and ('exams' not in x)]
        paths_frames = ['{}/{}'.format(path_exams, x) for x in frames_total if x not in frames_exists]
    else:
        paths_frames = ['{}/{}'.format(path_exams, x) for x in os.listdir(path_exams) if '.' not in x]
else:
    paths_frames.append('{}/{}'.format(path_exams, exam))

print(paths_frames[0:5])
print(len(paths_frames))

In [ ]:
def create_exam(list_frames, path_writer, size_writer, fps):
    out = cv2.VideoWriter(path_writer, cv2.VideoWriter_fourcc(*'DIVX'), fps, size_writer)

    for i in range(len(list_frames)):
        out.write(img_array[i])
    
    out.release()

In [ ]:
def make_path(path):
    try:
        os.mkdir(path)
    except FileExistsError:
        pass

In [ ]:
img_array = []
size = None

make_path(path_out)

for ex in paths_frames:    
    name_exam = ex[ex.rfind('/') + 1 : len(ex)]    
    path_writer = '{}/{}.mp4'.format(path_out, name_exam)
           
    path_original_exam = '{}/{}.mp4'.format(path_original_exams, name_exam)    
    original_exam = cv2.VideoCapture(path_original_exam)
    fps = original_exam.get(cv2.CAP_PROP_FPS)

    label = '{}/{}_label.csv'.format(path_labels, name_exam)
    print(label)
    dataset = pd.read_csv(label)

    number_files = int(len(os.listdir(ex)) / 5)
    for i in range(number_files):       
        path_final = 'final_{}.png'.format(i)
        path_binary = 'binary_{}.png'.format(i)
        path_process = 'process_{}.png'.format(i)
        path_original = 'original_{}.png'.format(i)
        path_histogram = 'histogram_{}.png'.format(i)
        path_img_process = 'img_process_{}.png'.format(i)

        im_final = cv2.imread('{}/{}'.format(ex, path_final))
        im_binary = cv2.imread('{}/{}'.format(ex, path_binary))
        im_process = cv2.imread('{}/{}'.format(ex, path_process))
        im_original = cv2.imread('{}/{}'.format(ex, path_original))
        im_histogram = cv2.imread('{}/{}'.format(ex, path_histogram))
        im_img_process = cv2.imread('{}/{}'.format(ex, path_img_process))

        im_pres = cv2.hconcat([im_original, im_process, im_binary, im_img_process])

        label = 'Frame=%d;Radius=%d;Center=(%d,%d);Eye=(%d)' % (dataset['frame'][i],
                                                                dataset['radius'][i],
                                                                dataset['center_x'][i],
                                                                dataset['center_y'][i],
                                                                0)

        cv2.putText(im_pres, label, (600, 30), cv2.FONT_HERSHEY_DUPLEX, 0.9, (170, 170, 0))

        img_array.append(im_pres)

        if size is None:
            height, width, _ = im_pres.shape
            size = (width, height)

    if not full_exam:
        create_exam(img_array, path_writer, size, fps)
        img_array.clear()

if full_exam:
    create_exam('full_exam', img_array)